# Submission 1

Kaggle team name: [55] Ship Happens

Course: TDT4173 Modern Machine Learning in Practice

Year: 2024

Group name: [55] Ship Happens

Group members:
| Name     | StudentId |
| -------- | ------- |
| Live Krohg  | 564347    |
| Ingvild Langeggen  | 564371     |
| Alexander Sundquist    | 132896    |

In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV
import matplotlib.pyplot as plt

In [2]:
#change the locations of files 
train = pd.read_csv('ais_train.csv', sep='|')
train['time'] = pd.to_datetime(train['time'])
train.info()
missing_values = train.isnull().sum()
print("Number of missing values in each column:\n", missing_values)
train.head()

test = pd.read_csv('ais_test.csv', sep=',')
test['time'] = pd.to_datetime(test['time'])
test.head()

vessels = pd.read_csv('vessels.csv', sep='|')
vessels.head()

ports = pd.read_csv('ports.csv', sep='|')
ports.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522065 entries, 0 to 1522064
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   time       1522065 non-null  datetime64[ns]
 1   cog        1522065 non-null  float64       
 2   sog        1522065 non-null  float64       
 3   rot        1522065 non-null  int64         
 4   heading    1522065 non-null  int64         
 5   navstat    1522065 non-null  int64         
 6   etaRaw     1522065 non-null  object        
 7   latitude   1522065 non-null  float64       
 8   longitude  1522065 non-null  float64       
 9   vesselId   1522065 non-null  object        
 10  portId     1520450 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(3)
memory usage: 127.7+ MB
Number of missing values in each column:
 time            0
cog             0
sog             0
rot             0
heading         0
navstat         0
etaRaw          0

,portId,name,portLocation,longitude,latitude,UN_LOCODE,countryName,ISO
0,61d36ed80a1807568ff9a064,Port of Algiers,Algiers,3.067222,36.773611,DZALG,Algeria,DZ
1,61d36ed80a1807568ff9a065,Port of Annaba,Annaba,7.772500,36.900556,DZAAE,Algeria,DZ
2,61d36edf0a1807568ff9a070,Port of Oran,Oran,-0.639722,35.712222,DZORN,Algeria,DZ
3,61d36ee00a1807568ff9a072,Port of Skikda,Skikda,6.905833,36.887500,DZSKI,Algeria,DZ
4,61d36ee10a1807568ff9a074,Port of Pago-Pago,Pago-Pago,-170.690556,-14.274167,ASPPG,American Samoa,AS


In [3]:
port_id_mapping = {port_id: idx for idx, port_id in enumerate(train['portId'].unique())}
train['portId'] = train['portId'].map(port_id_mapping)
ports['portId'] = ports['portId'].map(port_id_mapping)

vessel_id_mapping = {vessel_id: idx for idx, vessel_id in enumerate(train['vesselId'].unique())}
train['vesselId'] = train['vesselId'].map(vessel_id_mapping)
vessels['vesselId'] = vessels['vesselId'].map(vessel_id_mapping)
test['vesselId'] = test['vesselId'].map(vessel_id_mapping)


shipping_line_id_mapping = {shipping_line_id: idx for idx, shipping_line_id in enumerate(vessels['shippingLineId'].unique())}
vessels['shippingLineId'] = vessels['shippingLineId'].map(shipping_line_id_mapping)

In [4]:
train = train.merge(ports[['portId', 'latitude', 'longitude']], how='left', left_on='portId', right_on='portId', suffixes=('', '_port'))
train.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,latitude_port,longitude_port
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,0,0,-33.5875,-71.618889
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,1,1,8.9670,-79.533000
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,2,2,39.2325,-76.558889
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,3,3,-34.4625,150.899444
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,4,4,35.7830,-5.817000


In [5]:
train = train.merge(vessels[['vesselId', 'length', 'shippingLineId', 'breadth', 'GT']], on='vesselId', how='left')

missing_breadth = train[train['breadth'].isna()]

# Group by 'shippingLineId' and count missing 'breadth' values
missing_breadth_counts = (
    missing_breadth.groupby('shippingLineId')['breadth']
    .size()
    .reset_index(name='missing_breadth_count')
)

# Display only the shipping lines with missing values
print("Shipping lines with missing breadth values:")
print(missing_breadth_counts)

num_breadth_nan = train['breadth'].isna().sum()
print(f"Number of NaN values in maxSpeed: {num_breadth_nan}")

Shipping lines with missing breadth values:
   shippingLineId  missing_breadth_count
0               5                  14949
Number of NaN values in maxSpeed: 14949


In [6]:
mean_breadth_shipping_line_5 = train[train['shippingLineId'] == 5]['breadth'].mean()

train.loc[train['shippingLineId'] == 5, 'breadth'] = train.loc[train['shippingLineId'] == 5, 'breadth'].fillna(mean_breadth_shipping_line_5)

missing_count_after_imputation = train['breadth'].isnull().sum()
print(f"Missing Breadth values after imputation: {missing_count_after_imputation}")


Missing Breadth values after imputation: 0


In [7]:
train.info()
missing_values = train.isnull().sum()
print("Number of missing values in each column:\n", missing_values)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522065 entries, 0 to 1522064
Data columns (total 17 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   time            1522065 non-null  datetime64[ns]
 1   cog             1522065 non-null  float64       
 2   sog             1522065 non-null  float64       
 3   rot             1522065 non-null  int64         
 4   heading         1522065 non-null  int64         
 5   navstat         1522065 non-null  int64         
 6   etaRaw          1522065 non-null  object        
 7   latitude        1522065 non-null  float64       
 8   longitude       1522065 non-null  float64       
 9   vesselId        1522065 non-null  int64         
 10  portId          1522065 non-null  int64         
 11  latitude_port   1520450 non-null  float64       
 12  longitude_port  1520450 non-null  float64       
 13  length          1522065 non-null  float64       
 14  shippingLineId  15

In [8]:
train = train.dropna()

In [9]:

def create_five_day_windows(df):
    df['time'] = pd.to_datetime(df['time'])
    
    df = df.sort_values(by=['vesselId', 'time']).reset_index(drop=True)
    
    window_size_days = 5
    windows = []

    for vessel_id, group in df.groupby('vesselId'):
        group = group.sort_values(by='time')
        unique_dates = group['time'].dt.date.unique()

        for start_idx in range(len(unique_dates) - window_size_days + 1):
            start_date = unique_dates[start_idx]
            end_date = unique_dates[start_idx + window_size_days - 1]
            window = group[(group['time'].dt.date >= start_date) & (group['time'].dt.date <= end_date)]

            if len(window) == 0:
                continue

            reference_row = window.iloc[0]

            for _, row in window.iterrows():
                windows.append({
                    'vesselId': row['vesselId'],
                    'time': row['time'],
                    'latitude': row['latitude'],
                    'longitude': row['longitude'],
                    'cog_sin': row['cog_sin'],
                    'cog_cos': row['cog_cos'],
                    'cog' : row['cog'],
                    'sog': row['sog'],
                    'rot': row['rot'],
                    'under_way': row['under_way'],
                    'length' : row['length'],
                    'breadth': row['breadth'],
                    'GT': row['GT'],
                    # Features based on the first row in the window
                    'latitude_first': reference_row['latitude'],
                    'longitude_first': reference_row['longitude'],
                    'cog_sin_first': reference_row['cog_sin'],
                    'cog_cos_first': reference_row['cog_cos'],
                    'sog_first': reference_row['sog'],
                    'rot_first': reference_row['rot'],
                    'under_way_first': reference_row['under_way'],
                    'time_since_start': (row['time'] - reference_row['time']).total_seconds()
                })

    return pd.DataFrame(windows)

In [10]:
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a)) 
    r = 6371  # Radius of Earth in kilometers
    return r * c  # Distance in kilometers

### Feature engineering 

In [11]:
def feature_engineering(train):
    train = train.copy()
    train = train.sort_values(by=['vesselId', 'time']).reset_index(drop=True)

    features = pd.DataFrame()
    features['vesselId'] = train['vesselId']
    features['time'] = train['time'] 
    features['latitude'] = train['latitude']
    features['longitude'] = train['longitude']
    features['cog'] = train['cog']
    features['sog'] = train['sog']
    features['rot'] = train['rot']
    features['under_way'] = train['navstat'].isin([0, 8]).astype(int)
    features['cog_rad'] = np.radians(features['cog'])
    features['cog_sin'] = np.sin(features['cog_rad'])
    features['cog_cos'] = np.cos(features['cog_rad'])
    features['length'] = train['length']
    features['breadth'] = train['breadth']
    features['GT'] = train['GT']
    
    features = create_five_day_windows(features)

    features['latitude_port'] = train['latitude_port']
    features['longitude_port'] = train['longitude_port']

    features['distance_to_port'] = haversine(features['latitude'], features['longitude'], 
                                              features['latitude_port'], features['longitude_port'])
    
    features['month'] = features['time'].dt.month
    features['day'] = features['time'].dt.day
    features['hour'] = features['time'].dt.hour
    features['minute'] = features['time'].dt.minute

    return features

train_features = feature_engineering(train)

In [12]:
train_features = train_features.drop(columns=['time', 'rot_first'])

In [13]:
train_features.head()

,vesselId,latitude,longitude,cog_sin,cog_cos,cog,sog,rot,under_way,length,...,sog_first,under_way_first,time_since_start,latitude_port,longitude_port,distance_to_port,month,day,hour,minute
0,0,-34.74370,-57.85130,-0.970296,0.241922,284.0,0.7,0,1,199.0,...,0.7,1,0.0,-33.5875,-71.618889,1272.179414,1,1,0,0
1,0,-35.16787,-56.77210,0.998806,-0.048850,92.8,14.2,0,1,199.0,...,0.7,1,22123.0,-33.5875,-71.618889,1372.504428,1,1,6,9
2,0,-35.16863,-56.63185,0.999962,-0.008727,90.5,14.3,0,1,199.0,...,0.7,1,23874.0,-33.5875,-71.618889,1385.239318,1,1,6,38
3,0,-35.16805,-56.53190,0.999507,0.031411,88.2,14.3,0,1,199.0,...,0.7,1,25110.0,-33.5875,-71.618889,1394.309141,1,1,6,58
4,0,-35.16715,-56.45306,0.999560,0.029666,88.3,12.3,0,1,199.0,...,0.7,1,26131.0,-33.5875,-71.618889,1401.461476,1,1,7,15


In [14]:
def find_last_features(features):
    last_features = features.groupby('vesselId').last().reset_index()
    return last_features

last_features = find_last_features(train_features)
last_features.head()


,vesselId,latitude,longitude,cog_sin,cog_cos,cog,sog,rot,under_way,length,...,sog_first,under_way_first,time_since_start,latitude_port,longitude_port,distance_to_port,month,day,hour,minute
0,0,34.57936,128.99926,-0.662620,-0.748956,221.5,15.5,0,1,199.00,...,0.4,0,1340708.0,-34.190000,18.436944,13780.000982,5,1,12,41
1,1,1.24460,103.39997,-0.818150,0.575005,305.1,15.7,11,1,199.97,...,0.1,0,2195733.0,12.624444,100.919722,1294.559039,4,30,10,12
2,2,18.13873,-69.74863,0.069756,-0.997564,176.0,0.4,0,1,199.00,...,0.0,0,603974.0,53.344444,-6.209444,6652.930691,5,7,23,59
3,3,41.64055,143.29942,0.999123,0.041876,87.6,14.4,0,1,199.00,...,9.7,1,455585.0,17.633000,-101.550000,10654.769565,4,26,13,29
4,4,26.58710,121.27831,0.630676,0.776046,39.1,12.7,0,1,199.95,...,0.0,0,562766.0,53.524722,9.963333,8940.848077,5,7,12,28


### Train the model

In [15]:
y = train_features[['latitude', 'longitude']]  
X = train_features.drop(columns=['latitude', 'longitude', 'vesselId']) 
X.head()

,cog_sin,cog_cos,cog,sog,rot,under_way,length,breadth,GT,latitude_first,...,sog_first,under_way_first,time_since_start,latitude_port,longitude_port,distance_to_port,month,day,hour,minute
0,-0.970296,0.241922,284.0,0.7,0,1,199.0,32.0,57718,-34.7437,...,0.7,1,0.0,-33.5875,-71.618889,1272.179414,1,1,0,0
1,0.998806,-0.048850,92.8,14.2,0,1,199.0,32.0,57718,-34.7437,...,0.7,1,22123.0,-33.5875,-71.618889,1372.504428,1,1,6,9
2,0.999962,-0.008727,90.5,14.3,0,1,199.0,32.0,57718,-34.7437,...,0.7,1,23874.0,-33.5875,-71.618889,1385.239318,1,1,6,38
3,0.999507,0.031411,88.2,14.3,0,1,199.0,32.0,57718,-34.7437,...,0.7,1,25110.0,-33.5875,-71.618889,1394.309141,1,1,6,58
4,0.999560,0.029666,88.3,12.3,0,1,199.0,32.0,57718,-34.7437,...,0.7,1,26131.0,-33.5875,-71.618889,1401.461476,1,1,7,15


In [ ]:
best_model = RandomForestRegressor(max_depth=25, random_state=42, n_jobs=-1)
best_model.fit(X, y)


RandomForestRegressor(max_depth=25, n_jobs=-1, random_state=42)

Feature Importance 

In [ ]:
feature_importances = best_model.feature_importances_
features_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

features_df = features_df.sort_values(by='Importance', ascending=False)

print("Feature Importances:")
print(features_df)

Feature Importances:
             Feature  Importance
10   longitude_first    0.739001
15  time_since_start    0.115180
9     latitude_first    0.079880
8                 GT    0.013979
11     cog_sin_first    0.010984
6             length    0.006378
13         sog_first    0.005909
7            breadth    0.005353
12     cog_cos_first    0.004565
20               day    0.004045
3                sog    0.003519
0            cog_sin    0.003342
19             month    0.002147
18  distance_to_port    0.001563
2                cog    0.001162
1            cog_cos    0.000796
17    longitude_port    0.000600
16     latitude_port    0.000566
5          under_way    0.000512
14   under_way_first    0.000224
21              hour    0.000164
4                rot    0.000098
22            minute    0.000034


In [ ]:
X.head()
X.columns

Index(['cog_sin', 'cog_cos', 'cog', 'sog', 'rot', 'under_way', 'length',
       'breadth', 'GT', 'latitude_first', 'longitude_first', 'cog_sin_first',
       'cog_cos_first', 'sog_first', 'under_way_first', 'time_since_start',
       'latitude_port', 'longitude_port', 'distance_to_port', 'month', 'day',
       'hour', 'minute'],
      dtype='object')

Prepare test for predictions

In [ ]:

def prepare_test_for_predictions(test, last_features):
    test = test.copy()
    prepared_test = pd.DataFrame()

    last_features['year'] = 2024
    last_features['time'] = pd.to_datetime(
        last_features[['year', 'month', 'day', 'hour', 'minute']]
    )

    prepared_test['vesselId'] = test['vesselId']
    prepared_test['time'] = test['time']

    prepared_test = prepared_test.merge(last_features, on='vesselId', how='left', suffixes=('', '_last'))

    print(prepared_test.head())

    prepared_test['latitude_first'] = prepared_test['latitude']
    prepared_test['longitude_first'] = prepared_test['longitude']
    prepared_test['cog_sin_first'] = prepared_test['cog_sin']
    prepared_test['cog_cos_first'] = prepared_test['cog_cos']  
    prepared_test['sog_first'] = prepared_test['sog']
    prepared_test['under_way_first'] = prepared_test['under_way']
    prepared_test['time_since_start'] = (prepared_test['time'] - prepared_test['time_last']).dt.total_seconds()

    prepared_test['month'] = test['time'].dt.month
    prepared_test['day'] = test['time'].dt.day
    prepared_test['hour'] = test['time'].dt.hour
    prepared_test['minute'] = test['time'].dt.minute

    prepared_test.drop(['time', 'time_last', 'year'], axis=1, inplace=True)

    prepared_test = prepared_test[['cog_sin', 'cog_cos', 'cog', 'sog', 'rot', 'under_way', 'length', 'breadth',
       'GT', 'latitude_first', 'longitude_first', 'cog_sin_first',
       'cog_cos_first', 'sog_first', 'under_way_first', 'time_since_start',
       'latitude_port', 'longitude_port', 'distance_to_port', 'month', 'day',
       'hour', 'minute'
                                ]]
    

    return prepared_test

test_df = prepare_test_for_predictions(test, last_features)
print(test_df.head())
print(test_df.shape)


   vesselId                time  latitude  longitude   cog_sin   cog_cos  \
0       412 2024-05-08 00:03:16  31.14647  -81.49789  0.006981 -0.999976   
1       373 2024-05-08 00:06:17  14.81694  120.29625  0.417867  0.908508   
2       181 2024-05-08 00:10:02  38.27895   10.78280  0.139173  0.990268   
3         8 2024-05-08 00:10:34 -43.53785  172.83522 -0.625243  0.780430   
4        65 2024-05-08 00:12:27  48.53320   -6.12003 -0.933580  0.358368   

     cog   sog  rot  under_way  ...  time_since_start  latitude_port  \
0  179.6   0.0    0          0  ...          516824.0            NaN   
1   24.7   0.0    0          0  ...          776687.0            NaN   
2    8.0  18.7    0          1  ...          426241.0            NaN   
3  321.3   0.1    0          0  ...          430552.0      53.952222   
4  291.0   0.3    0          0  ...          430370.0      43.555833   

   longitude_port  distance_to_port  month  day  hour  minute  year  \
0             NaN               NaN    

In [ ]:
predictions = best_model.predict(test_df)
print(predictions)
print(predictions.shape)

[[  31.15447708  -81.47341648]
 [  14.6999973   120.5835811 ]
 [  38.79095657   10.92009542]
 ...
 [  36.5663055  -110.7916724 ]
 [  57.22327693   18.20169838]
 [  40.85143165   -6.10179084]]
(51739, 2)


In [ ]:
predictions_df = pd.DataFrame(predictions, columns=['latitude_predicted', 'longitude_predicted'])
predictions_df['ID'] = range(len(predictions_df))
predictions_df = predictions_df[['ID', 'longitude_predicted', 'latitude_predicted']]

predictions_df.to_csv('data/submissions/predictions_rf_6.csv', index=False)
# predictions_df.to_csv('notebook_x_rf.csv', index=False)

In [ ]:
merged_df = pd.merge(test, predictions_df, on='ID', how='left')

print("Columns in merged_df before dropping:", merged_df.columns.tolist())

columns_to_drop = ['ID', 'scaling_factor']
for col in columns_to_drop:
    if col in merged_df.columns:
        merged_df.drop(col, axis=1, inplace=True)

print("Columns in merged_df after dropping:", merged_df.columns.tolist())
print(merged_df.head())

Columns in merged_df before dropping: ['ID', 'vesselId', 'time', 'scaling_factor', 'longitude_predicted', 'latitude_predicted']
Columns in merged_df after dropping: ['vesselId', 'time', 'longitude_predicted', 'latitude_predicted']
   vesselId                time  longitude_predicted  latitude_predicted
0       412 2024-05-08 00:03:16           -81.473416           31.154477
1       373 2024-05-08 00:06:17           120.583581           14.699997
2       181 2024-05-08 00:10:02            10.920095           38.790957
3         8 2024-05-08 00:10:34           172.944165          -43.417744
4        65 2024-05-08 00:12:27            -3.105527           48.355467


In [ ]:
merged_df.to_csv('data/submissions/plotting_boats_6_rf.csv')